## Create a knowledgebase using prepositional logic and prove the given query using resolution
Akshay Mittur, 1BM18CS010

In [27]:
def disjunctify(clauses):
    disjuncts = []
    for clause in clauses:
        disjuncts.append(tuple(clause.split('v')))
    return disjuncts

def getResolvant(ci, cj, di, dj):
    resolvant = list(ci) + list(cj)
    resolvant.remove(di)
    resolvant.remove(dj)
    return tuple(resolvant)

def resolve(ci, cj):
    for di in ci:
        for dj in cj:
            if di == '~' + dj or dj == '~' + di:
                return getResolvant(ci, cj, di, dj)

In [28]:
def checkResolution(clauses, query):
    clauses += [query if query.startswith('~') else '~' + query]
    proposition = '^'.join(['(' + clause + ')' for clause in clauses])
    print(f'Trying to prove {proposition} by contradiction....')
    
    clauses = disjunctify(clauses)
    resolved = False
    new = set()
    
    while not resolved:
        n = len(clauses)
        pairs = [(clauses[i], clauses[j]) for i in range(n) for j in range(i + 1, n)]
        for (ci, cj) in pairs:
            resolvant = resolve(ci, cj)
            if not resolvant:
                resolved = True
                break
            new = new.union(set(resolvents))
        if new.issubset(set(clauses)):
            break
        for clause in new:
            if clause not in clauses:
                clauses.append(clause)
        
    if resolved:
        print('Knowledge Base entails the query, proved by resolution')
    else:
        print("Knowledge Base doesn't entail the query, no empty set produced after resolution")

In [29]:
#Test Case 1
clauses = input('Enter the clauses ').split()
query = input('Enter the query: ')
checkResolution(clauses, query)

Enter the clauses AvB BvC ~C
Enter the query: B
Trying to prove (AvB)^(BvC)^(~C)^(~B) by contradiction....
Knowledge Base entails the query, proved by resolution


In [32]:
#Test Case 2
clauses = input('Enter the clauses ').split()
query = input('Enter the query: ')
checkResolution(clauses, query)

Enter the clauses ~Qv~PvR ~Q^P Q
Enter the query: Q
Trying to prove (~Qv~PvR)^(~Q^P)^(Q)^(~Q) by contradiction....
Knowledge Base entails the query, proved by resolution
